In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense
#from keras.layers import LSTR
from keras.models import Sequential
from keras.models import load_model
#import tensorflow as tf
import matplotlib.pyplot as plt
#from sklearn.linear_model import SGDRegressor as SGD
import joblib
#from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# HyperParameters and Model Architecture
N_LAYERS = 10
N_NODES = 25
TEST_SIZE = 0.25
DAYS_TO_CLIP = 1
EPOCHS = 2

#tf.random.set_seed(21)

In [3]:
# specify location of time series data
file_path = '../data/interim/time_series.csv'

def read_data(file):
    """
    Read csv data from the specified file location.
    """
    df = pd.read_csv(file, index_col='Date')
    return df


# read time series data
time_series_df = read_data(file_path)

# test_df will be used to test on recent market performance
test_df = read_data(file_path) 


def clip_recent_days(df, n_days):
    """
    remove recent days from data frame
    """
    return(df[:-n_days])


# clip most recent days
time_series_df = clip_recent_days(time_series_df, DAYS_TO_CLIP)

# inspect the time series dataframe
time_series_df

#time_series_df = pd.read_csv('../data/interim/time_series.csv', index_col='Date')
#test_df = time_series_df.copy()
#time_series_df = time_series_df[:-DAYS_TO_CLIP]
#time_series_df

,back_5,back_4,back_3,back_2,back_1,Adj Close
Date,,,,,,
1962-01-02,NaN,NaN,NaN,NaN,NaN,0.019803
1962-01-03,NaN,NaN,NaN,NaN,0.019803,-0.009852
1962-01-04,NaN,NaN,NaN,0.019803,-0.009852,-0.020001
1962-01-05,NaN,NaN,0.019803,-0.009852,-0.020001,0.002522
1962-01-08,NaN,0.019803,-0.009852,-0.020001,0.002522,0.002516
...,...,...,...,...,...,...
2020-03-26,-0.028022,0.105866,0.189679,0.217677,0.128803,-0.108411
2020-03-27,0.105866,0.189679,0.217677,0.128803,-0.108411,-0.061875
2020-03-30,0.189679,0.217677,0.128803,-0.108411,-0.061875,-0.020835


In [4]:
time_series_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14663 entries, 1962-01-02 to 2020-04-01
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   back_5     14658 non-null  float64
 1   back_4     14659 non-null  float64
 2   back_3     14660 non-null  float64
 3   back_2     14661 non-null  float64
 4   back_1     14662 non-null  float64
 5   Adj Close  14663 non-null  float64
dtypes: float64(6)
memory usage: 801.9+ KB


In [5]:
def get_tomorrow(df):
    tomorrow = df.tail(1)
    predictors = tomorrow.iloc[:,:5]
    target = tomorrow.iloc[:,-1]
    return predictors, target
    
tomorrows_predictors, tomorrows_target = get_tomorrow(time_series_df)

get_tomorrow(time_series_df)
    
#tomorrow = time_series_df.tail(1)
#tomorrows_predictors = tomorrow.iloc[:,:5]
#tomorrows_predictors

(              back_5    back_4    back_3    back_2    back_1
 Date                                                        
 2020-04-01  0.128803 -0.108411 -0.061875 -0.020835 -0.131981,
 Date
 2020-04-01   -0.058528
 Name: Adj Close, dtype: float64)

In [6]:
def format_predictors_and_targets(df):
    
    df = df.dropna()
    
    predictors = df[['back_5', 'back_4', 'back_3', 'back_2', 'back_1']].values
    assert type(predictors) is np.ndarray
    
    n_cols = predictors.shape[1]
    
    targets = df[['Adj Close']].values
    assert type(targets) is np.ndarray
    
    return predictors, targets, n_cols


#time_series_df = time_series_df.dropna()
#time_series_df.info()

In [7]:
# create array of predictors
#predictors = time_series_df[['back_5', 'back_4', 'back_3', 'back_2', 'back_1']].values
#n_cols = predictors.shape[1]
#type(predictors)

In [8]:
# create array of targets
#target = time_series_df[['Adj Close']].values
#type(target)

In [9]:
predictors, targets, n_cols = format_predictors_and_targets(time_series_df)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=TEST_SIZE)

In [11]:
def build_sequential(n_nodes, n_layers, n_cols):
    
    model = Sequential()
    
    model.add(Dense(n_nodes, activation='relu', input_shape=(n_cols,)))
    
    for i in range(n_layers-1):
        model.add(Dense(n_nodes, activation='relu'))
    
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

model = build_sequential(N_NODES, N_LAYERS, n_cols)
model.summary()
        
    

#initialize a sequential keras model
#model = Sequential()

# build with N_LAYERS N_NODES each
#model.add(Dense(N_NODES, activation='relu', input_shape=(n_cols,)))
#for i in range(N_LAYERS-1):
#    model.add(Dense(N_NODES, activation='relu'))


# build final layer that will contain the output prediction
#model.add(Dense(1))

# compile the model with ADAM
# will adjust the learning rate as it does gradient descent
# MSE loss function is best for regression problems
#model.compile(optimizer='adam', loss='mean_squared_error')

# verify model structure
#model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                150       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_4 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_5 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_6 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_7 (Dense)              (None, 25)               

# fit the model

In [12]:
 # fit the model
# apply back propagation and gradient descent
model.fit(X_train, y_train, use_multiprocessing=True, epochs=EPOCHS)

Epoch 1/2
10993/10993 [==============================] - 1s 117us/step - loss: 4.2538e-04
Epoch 2/2
10993/10993 [==============================] - 1s 81us/step - loss: 4.2506e-04


In [13]:
# save and load model
model.save('../models/keras.h5')
model = load_model('../models/keras.h5')

# verify model structure
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                150       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_4 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_5 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_6 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_7 (Dense)              (None, 25)               

In [14]:
predictions = model.predict(X_test)
output = predictions
output = pd.DataFrame(output)
output.to_csv('../data/interim/keras_dense_predictions.csv')
def revert_exp(predictions, y_test):
    predictions = np.exp(predictions)
    y_test_exp = np.exp(y_test)
    return predictions, y_test_exp

# min(y_test_exp), max(y_test_exp)
predictions, y_test_exp = revert_exp(predictions, y_test)

In [15]:
scores = [53.964194, 45.865303, 53.878943, 53.793691]

In [16]:
#### this is very good

def get_accuracy(y, pred):
    
    #scale and shift binary results
    # -1 -> stock went down
    # +1 -> stock increased or stayed the same
    y = ((y>=1)*2)-1
    pred = ((pred>=1)*2)-1
    
    # stocks move in the same direction when a_i*b_i is positive
    accuracy = (np.sum((y*pred)>=0)/len(y))*100
    
    print("Predicting change in stock price with %f%s accuracy" % (accuracy,'%'))
    
    return accuracy

scores.append(get_accuracy(y_test_exp, predictions))

Predicting change in stock price with 53.233288% accuracy


In [17]:
int('Stop here')

ValueError: invalid literal for int() with base 10: 'Stop here'

# Trading dates where COVID-19 affected stock market need to be excluded. if included, accuracy drops to ~48%

# Now that we can make and evaluate predictions, It's time for some Cross-Validation to improve the Neural Network's hyperparameters.

In [ ]:
DAYS_CLIPPED = [1, 5, 10, 15, 20, 25, 30, 35, 40]
TEST_SIZES = [0.15, 0.20, 0.25, 0.35, 0.40]
NODES = [5, 25, 50, 75, 100, 150, 200, 500]
LAYERS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
EPOCHS = [1, 2, 3, 4, 5, 6]


In [ ]:
len(DAYS_CLIPPED) * len(TEST_SIZES) * len(NODES) * len(layers) * len(epochs)

#21,600

In [ ]:
def run_cross_validation():
    #best_layers = 0
    #best_nodes = 0
    #best_test_size = 0
    #best_days_clipped = 0
    #best_epochs = 0
    best_mean_accuracy = 0
    loop = 0
    
    # read_data
    time_series_df = read_data(file_path)
    
    
    for DAYS_TO_CLIP in DAYS_CLIPPED:
        
        time_series_df = clip_recent_days(time_series_df, DAYS_TO_CLIP)
        predictors, targets, n_cols = format_predictors_and_targets(time_series_df)
        
        
        for TEST_SIZE in TEST_SIZES:
            
            X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=TEST_SIZE)
            
            
            for N_NODES in NODES:
                for N_LAYERS in LAYERS:
                    
                    model = build_sequential(N_NODES, N_LAYERS, n_cols)
                    
                    
                    for N_EPOCHS in EPOCHS:
                        
                        accuracies = []
                        
                        for i in range(10):
                            
                            model.fit(X_train, y_train, use_multiprocessing=True, epochs=N_EPOCHS)
                            predictions = model.predict(X_test)
                            predictions, y_test_exp = revert_exp(predictions, y_test)
                            accuracies.append(get_accuracy(y_test_exp, predictions))
                            loop += 1
                            print('PROGRESS: ', loop)
                        
                        mean_accuracy = np.mean(accuracies)
                        
                        if mean_accuracy>best_mean_accuracy:
                            hyperparameters = {
                                'layers': N_LAYERS,
                                'nodes': N_NODES,
                                'test_size': TEST_SIZE,
                                'days_clipped': DAYS_TO_CLIP,
                                'epochs': N_EPOCHS,
                                'mean_accuracy': mean_accuracy
                            }
                            #best_layers = N_LAYERS
                            #best_nodes = N_NODES
                            #best_test_size = TEST_SIZE
                            #best_days_clipped = DAYS_TO_CLIP
                            #best_epochs = N_EPOCHS
                            best_mean_accuracy = mean_accuracy
                            best_model = model
                        print(hyperparameters)
                            
    return model, hyperparameters

model, params = run_cross_validation()

                                                      
            
        
    
    
    

In [ ]:
scores

In [ ]:
raw_data = pd.read_csv('../data/raw/raw.csv', index_col=['Date'])
raw_data

In [ ]:
viz_df = raw_data[['Adj Close']]
viz_df.columns = ['price']


viz_df = pd.merge(viz_df, time_series_df, left_index=True, right_index=True, how='outer')

viz_df = viz_df.dropna()

X_viz = viz_df[['back_5', 'back_4', 'back_3', 'back_2', 'back_1']].values

preds_viz = model.predict(X_viz)
preds_viz = np.exp(preds_viz)
viz_df['predicted_price'] = preds_viz
viz_df['predicted_price'] = viz_df['price'] * viz_df['predicted_price']
viz_df

In [ ]:
def predict_tomorrow():
    X = tomorrows_predictors.values
    prediction = model.predict(X)
    prediction = np.exp(prediction)
    adj_close = raw_data[['Adj Close']]
    prices = adj_close.tail(1)
    todays_price = float(prices.values[0])
    tomorrows_price = float(todays_price) * float(prediction)
    is_increase = bool(prediction>=1)
    return todays_price, tomorrows_price, is_increase

print(predict_tomorrow())


In [ ]:
month = viz_df.loc['2018-05-01':'2018-06-30']
month

# This next plot will be very nice becuase it will show the model's ability to make predictions on the training data. 

In [ ]:
_ = plt.figure(figsize=(15,15))
_ = plt.plot(month['price'],  marker='.', alpha=0.9)
_ = plt.plot(month['predicted_price'], marker='.', alpha=0.75)
_ = plt.legend(['Actual Price', 'Predictions'])
_ = plt.xticks(rotation=90)
plt.show()

In [ ]:
test_df = test_df.tail(40).head(30)
X_test = test_df[['back_5', 'back_4', 'back_3', 'back_2', 'back_1']].values
test_preds = model.predict(X_test)
test_preds = np.exp(test_preds)
test_df['price'] = raw_data['Adj Close']
test_df['predictions'] = test_preds
test_df['pred'] = (test_df['predictions'] * test_df['price'])
test_df

In [ ]:
_ = plt.figure(figsize=(15,15))
_ = plt.plot(test_df['price'], marker='.')
_ = plt.plot(test_df['pred'], marker='.')
_ = plt.xticks(rotation=90)
_ = plt.legend(['Actual Price', 'Predictions'])
plt.show()

In [ ]:
int('stop notebook execution')

In [ ]:
def revert_data(predictions):
    
    # undo scale
    #scaler = joblib.load('../models/MinMaxScaler.save')
    #unscaled_deltas = scaler.inverse_transform(predictions)
    
    # undo log
    #inv_log = np.exp(unscaled_deltas)
    inv_log = np.exp(predictions)
    
    return inv_log
x = revert_data(predictions)
(x.min(), x.max())

In [ ]:

scaler = joblib.load('../models/MinMaxScaler.save')
unscaled_deltas = scaler.inverse_transform(predictions)
unscaled_deltas

scaler.inverse_transform([[0.81, 0.12, 3.3], [0,1, -2]])

In [ ]:
x1 = np.array([[ 0.12417709, -0.08672323,  0.88525218],[-0.12340155,  0.1822511 , -0.73470684]])
scaler.transform(x1)

In [ ]:
inv_log = np.exp(predictions)
inv_log

In [ ]:
inv_log.min(), inv_log.max()

In [ ]:
_ = plt.figure(figsize=(14,7))
_ = plt.plot(predictions, linestyle='None', marker='.')
plt.show()

In [ ]:
def get_Dense_model(input_shape):
    #initialize a sequential keras model
    model = Sequential()

    # build three layers with 100 nodes each
    model.add(Dense(100, activation='relu', input_shape=input_shape))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))

    # build final layer that will contain the output prediction
    model.add(Dense(1))

In [ ]:
def get_LSTR_model(input_shape):
    # Matt and Druv's architecture
    #initialize a sequential keras model
    model = Sequential()

    # build three layers with 100 nodes each
    model.add(LSTR(50, return_sequence=True, input_shape=input_shape))
    model.add(LSTR(50, return_sequence=True)
    model.add(LSTR(50))

    # build final layer that will contain the output prediction
    model.add(Dense(1))
    
    mode.complile(loss='mse', optimizer='adam')
    
              

In [ ]:
lr_to_test = [0.00001, 0.01, 1]
input_shape = (n_cols,)

# loop over learning rates
for lr in lr_to_test:
    model = get_new_model(input_shape)
    
    # use Stochastic Gradient Descent
    optimizer = SGD(lr=lr)
    
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    mse = sum((predictions - y_test)**2)
    
    print(lr, mse)